# Imports

In [40]:
import requests
import json
import pandas as pd
import time
from time import sleep
import random
from random import randint
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', None)


# Extract info from website :

In [41]:
%%time

pages = []
for i in range(1,5000):

    response = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params = {
        'currency_code':'EUR',
        'grape_filter':'varietal',
        'min_rating':'3.6',
        'order_by':'ratings_average',
        'order':'desc',
        'wine_type_ids[]':'2',
        'page':i,
        'language':'en'
    },
    headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }
)

    #store response into "pages" list
    pages.append(response)

    # respectful nap:
    wait_time = randint(1,2000)
    sleep(wait_time/1000)

CPU times: user 56.1 s, sys: 11.9 s, total: 1min 7s
Wall time: 2h 25min 17s


In [42]:
len(pages)

4999

In [43]:
# Nr. of wines
response.json()['explore_vintage']['records_matched']

10831

In [44]:
response.json()['explore_vintage']['matches'][0].keys()

dict_keys(['vintage', 'price', 'prices'])

## Vintage

In [45]:
all_wines = []
vintage = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        vintage.append(all_wines[i][j]['vintage'])

In [46]:
vintage_df = pd.DataFrame(vintage)
vintage_df.head(3)

,id,seo_name,name,statistics,image,wine,year,grapes,has_valid_ratings,top_list_rankings
0,164101692,egon-muller-scharzhof-riesling-le-gallais-wilt...,Egon Müller - Scharzhof Riesling 'Le Gallais' ...,"{'status': 'Normal', 'ratings_count': 28, 'rat...",{'location': '//images.vivino.com/thumbs/vxTQZ...,"{'id': 1467543, 'name': 'Riesling 'Le Gallais'...",2019,None,True,NaN
1,150679517,joseph-drouhin-montrachet-grand-cru-marquis-de...,Joseph Drouhin Montrachet Grand Cru Marquis de...,"{'status': 'Normal', 'ratings_count': 120, 'ra...",{'location': '//images.vivino.com/thumbs/1QoFU...,"{'id': 1274124, 'name': 'Montrachet Grand Cru ...",2017,None,True,NaN
2,161945961,keller-kirchspiel-riesling-gg-2019,Keller Kirchspiel Riesling GG 2019,"{'status': 'Normal', 'ratings_count': 48, 'rat...",{'location': '//images.vivino.com/thumbs/3hM2p...,"{'id': 1186645, 'name': 'Kirchspiel Riesling G...",2019,None,True,NaN


In [47]:
vintage_df['id'].nunique()

2024

In [48]:
#vintage_df['top_list_rankings'].value_counts(dropna = False)

#### Remove unnecessary columns

 - 'statistics' and 'wine' info will be extracted separetely, therefore I will drop these columns as well;
 - 'image' and 'top_list_rankings' are not required for the model
 - 'grapes' only has None values. This info is captured under wines section
 - 'has_valid_ratings' is always 'True'

In [49]:
vintage_df = vintage_df.drop(['statistics','image','wine','grapes','has_valid_ratings','top_list_rankings'], axis = 1)
vintage_df

,id,seo_name,name,year
0,164101692,egon-muller-scharzhof-riesling-le-gallais-wilt...,Egon Müller - Scharzhof Riesling 'Le Gallais' ...,2019
1,150679517,joseph-drouhin-montrachet-grand-cru-marquis-de...,Joseph Drouhin Montrachet Grand Cru Marquis de...,2017
2,161945961,keller-kirchspiel-riesling-gg-2019,Keller Kirchspiel Riesling GG 2019,2019
3,156470438,louis-latour-corton-charlemagne-grand-cru-2018,Louis Latour Corton-Charlemagne Grand Cru 2018,2018
4,162261533,domaine-faiveley-bienvenues-batard-montrachet-...,Domaine Faiveley Bienvenues-Bâtard-Montrachet ...,2018
...,...,...,...,...
124970,160509564,minuty-blanc-et-or-2019,Minuty Blanc et Or 2019,2019
124971,142657578,peter-jakob-kuhn-lenchen-riesling-spatlese-2015,Peter Jakob Kühn Lenchen Riesling Spätlese 2015,2015
124972,29929823,michel-redde-et-fils-les-cornets-blanc-fume-de...,Michel Redde Les Cornets Blanc Fumé de Pouilly...,2014
124973,163002580,herdade-aldeia-de-cima-alyantiju-branco-2018,Herdade Aldeia de Cima Alyantiju Branco 2018,2018


## Statistics

In [50]:
all_wines = []
statistics = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        statistics.append(all_wines[i][j]['vintage']['statistics'])

In [51]:
statistics_df = pd.DataFrame(statistics)
statistics_df.head(3)

,status,ratings_count,ratings_average,labels_count,wine_ratings_count,wine_ratings_average,wine_status
0,Normal,28,4.8,111,216,4.5,Normal
1,Normal,120,4.8,675,1870,4.6,Normal
2,Normal,48,4.7,196,1652,4.5,Normal


In [52]:
len(statistics_df['status'])

124975

#### Remove unnecessary columns

In [53]:
# for col in statistics_df.columns:
#     #print(customer_df[col].unique)
#     print('Name: {}'.format(statistics_df[col].name))    # column name
#     print('Type: {}'.format(statistics_df[col].dtype))   # column type
#     print('Unique values: {}'.format(len(statistics_df[col].unique())))   # column unique values
#     print('% Unique values: {}'.format(round(len(statistics_df[col].unique())/len(statistics_df)*100,1)))  # percentage
#     print('\n----------')
    

In [54]:
statistics_df = statistics_df.drop(['status','labels_count','wine_ratings_count','wine_ratings_average','wine_status'], axis = 1)
statistics_df

,ratings_count,ratings_average
0,28,4.8
1,120,4.8
2,48,4.7
3,234,4.7
4,33,4.7
...,...,...
124970,54,4.2
124971,54,4.2
124972,54,4.2
124973,54,4.2


## Wine info

In [55]:
all_wines = []
wine = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        wine.append(all_wines[i][j]['vintage']['wine'])

In [56]:
wine_df = pd.DataFrame(wine)

wine_df.columns = 'wine_' + wine_df.columns
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, 'ra...","{'id': 186, 'seo_name': 'german-riesling', 're...",True
1,1274124,Montrachet Grand Cru Marquis de Laguiche,montrachet-grand-cru-marquis-de-laguiche,2,0,False,"{'id': 3041, 'name': 'Montrachet Grand Cru', '...","{'id': 1358, 'name': 'Joseph Drouhin', 'seo_na...","{'structure': {'acidity': 3.1058955, 'fizzines...","{'status': 'Normal', 'ratings_count': 1870, 'r...","{'id': 285, 'seo_name': 'burgundy-cote-de-beau...",True
2,1186645,Kirchspiel Riesling GG,kirchspiel-riesling-gg,2,0,False,"{'id': 480, 'name': 'Rheinhessen', 'name_en': ...","{'id': 13734, 'name': 'Keller', 'seo_name': 'k...","{'structure': {'acidity': 4.518922, 'fizziness...","{'status': 'Normal', 'ratings_count': 1652, 'r...","{'id': 186, 'seo_name': 'german-riesling', 're...",True


### wine_region

In [57]:
#Flat columns
wine_region = pd.json_normalize(wine_df['wine_region'])
wine_region.columns = 'wine_region_' + wine_region.columns
wine_region.shape

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_region], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_region'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 33)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, 'ra...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,3150.0,Wiltingen,,wiltingen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",NaN,NaN,NaN
1,1274124,Montrachet Grand Cru Marquis de Laguiche,montrachet-grand-cru-marquis-de-laguiche,2,0,False,"{'id': 3041, 'name': 'Montrachet Grand Cru', '...","{'id': 1358, 'name': 'Joseph Drouhin', 'seo_na...","{'structure': {'acidity': 3.1058955, 'fizzines...","{'status': 'Normal', 'ratings_count': 1870, 'r...","{'id': 285, 'seo_name': 'burgundy-cote-de-beau...",True,3041.0,Montrachet Grand Cru,Le Montrachet,montrachet-grand-cru,NaN,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN
2,1186645,Kirchspiel Riesling GG,kirchspiel-riesling-gg,2,0,False,"{'id': 480, 'name': 'Rheinhessen', 'name_en': ...","{'id': 13734, 'name': 'Keller', 'seo_name': 'k...","{'structure': {'acidity': 4.518922, 'fizziness...","{'status': 'Normal', 'ratings_count': 1652, 'r...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,480.0,Rheinhessen,,rheinhessen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",//images.vivino.com/regions/backgrounds/ogLP94...,//thumbs.vivino.com/region_backgrounds/ogLP94y...,//thumbs.vivino.com/region_backgrounds/ogLP94y...


### wine_winery

In [58]:
#Flat columns
wine_winery = pd.json_normalize(wine_df['wine_winery'])
wine_winery.columns = 'wine_winery_' + wine_winery.columns
# print(wine_winery.shape)
# wine_winery

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_winery], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 42)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, 'ra...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,3150.0,Wiltingen,,wiltingen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",NaN,NaN,NaN,2197,Egon Müller - Scharzhof,egon-muller-scharzhof,0,NaN,NaN,NaN,NaN,NaN
1,1274124,Montrachet Grand Cru Marquis de Laguiche,montrachet-grand-cru-marquis-de-laguiche,2,0,False,"{'id': 3041, 'name': 'Montrachet Grand Cru', '...","{'id': 1358, 'name': 'Joseph Drouhin', 'seo_na...","{'structure': {'acidity': 3.1058955, 'fizzines...","{'status': 'Normal', 'ratings_count': 1870, 'r...","{'id': 285, 'seo_name': 'burgundy-cote-de-beau...",True,3041.0,Montrachet Grand Cru,Le Montrachet,montrachet-grand-cru,NaN,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,1358,Joseph Drouhin,joseph-drouhin,0,NaN,NaN,NaN,NaN,NaN
2,1186645,Kirchspiel Riesling GG,kirchspiel-riesling-gg,2,0,False,"{'id': 480, 'name': 'Rheinhessen', 'name_en': ...","{'id': 13734, 'name': 'Keller', 'seo_name': 'k...","{'structure': {'acidity': 4.518922, 'fizziness...","{'status': 'Normal', 'ratings_count': 1652, 'r...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,480.0,Rheinhessen,,rheinhessen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",//images.vivino.com/regions/backgrounds/ogLP94...,//thumbs.vivino.com/region_backgrounds/ogLP94y...,//thumbs.vivino.com/region_backgrounds/ogLP94y...,13734,Keller,keller,0,NaN,NaN,NaN,NaN,NaN


### wine_taste

In [59]:
#Flat columns
wine_taste = pd.json_normalize(wine_df['wine_taste'])
wine_taste.columns = 'wine_taste_' + wine_taste.columns
#print(wine_taste.shape)
#wine_taste

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_taste], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 51)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_flavor,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, 'ra...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,3150.0,Wiltingen,,wiltingen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",NaN,NaN,NaN,2197,Egon Müller - Scharzhof,egon-muller-scharzhof,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'citrus_fruit', 'stats': {'count': ...",4.795868,NaN,3.078479,2.826866,NaN,33.0,14.0,NaN
1,1274124,Montrachet Grand Cru Marquis de Laguiche,montrachet-grand-cru-marquis-de-laguiche,2,0,False,"{'id': 3041, 'name': 'Montrachet Grand Cru', '...","{'id': 1358, 'name': 'Joseph Drouhin', 'seo_na...","{'structure': {'acidity': 3.1058955, 'fizzines...","{'status': 'Normal', 'ratings_count': 1870, 'r...","{'id': 285, 'seo_name': 'burgundy-cote-de-beau...",True,3041.0,Montrachet Grand Cru,Le Montrachet,montrachet-grand-cru,NaN,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,1358,Joseph Drouhin,joseph-drouhin,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'oak', 'stats': {'count': 111, 'sco...",3.105895,NaN,3.834148,1.982433,NaN,152.0,132.0,NaN
2,1186645,Kirchspiel Riesling GG,kirchspiel-riesling-gg,2,0,False,"{'id': 480, 'name': 'Rheinhessen', 'name_en': ...","{'id': 13734, 'name': 'Keller', 'seo_name': 'k...","{'structure': {'acidity': 4.518922, 'fizziness...","{'status': 'Normal', 'ratings_count': 1652, 'r...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,480.0,Rheinhessen,,rheinhessen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",//images.vivino.com/regions/backgrounds/ogLP94...,//thumbs.vivino.com/region_backgrounds/ogLP94y...,//thumbs.vivino.com/region_backgrounds/ogLP94y...,13734,Keller,keller,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'earth', 'stats': {'count': 84, 'sc...",4.518922,NaN,3.303919,1.702501,NaN,246.0,111.0,NaN


#### wine_taste_flavor

In [60]:
#Flat columns
wine_flavor = pd.json_normalize(wine_df['wine_taste_flavor'])


# #Flat the first 4 columns from flavor table. This are most voted tastes by users
wine_flavor_1 = pd.json_normalize(wine_flavor[0])
wine_flavor_1.columns = 'wine_taste_flavor_1' + wine_flavor_1.columns
wine_flavor_2 = pd.json_normalize(wine_flavor[1])
wine_flavor_2.columns = 'wine_taste_flavor_2' + wine_flavor_2.columns
wine_flavor_3 = pd.json_normalize(wine_flavor[2])
wine_flavor_3.columns = 'wine_taste_flavor_3' + wine_flavor_3.columns
wine_flavor_4 = pd.json_normalize(wine_flavor[3])
wine_flavor_4.columns = 'wine_taste_flavor_4' + wine_flavor_4.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_flavor_1, wine_flavor_2, wine_flavor_3, wine_flavor_4], axis=1)
wine_df = wine_df.drop('wine_taste_flavor', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1primary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, 'ra...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,3150.0,Wiltingen,,wiltingen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",NaN,NaN,NaN,2197,Egon Müller - Scharzhof,egon-muller-scharzhof,0,NaN,NaN,NaN,NaN,NaN,4.795868,NaN,3.078479,2.826866,NaN,33.0,14.0,NaN,citrus_fruit,"[{'id': 243, 'name': 'lemon', 'count': 7}, {'i...",10.0,1500.0,NaN,earth,"[{'id': 218, 'name': 'honey', 'count': 8}, {'i...",10.0,1258.0,NaN,tropical_fruit,"[{'id': 329, 'name': 'pineapple', 'count': 7},...",9.0,1500.0,NaN,tree_fruit,"[{'id': 271, 'name': 'melon', 'count': 1}, {'i...",4.0,500.0,NaN
1,1274124,Montrachet Grand Cru Marquis de Laguiche,montrachet-grand-cru-marquis-de-laguiche,2,0,False,"{'id': 3041, 'name': 'Montrachet Grand Cru', '...","{'id': 1358, 'name': 'Joseph Drouhin', 'seo_na...","{'structure': {'acidity': 3.1058955, 'fizzines...","{'status': 'Normal', 'ratings_count': 1870, 'r...","{'id': 285, 'seo_name': 'burgundy-cote-de-beau...",True,3041.0,Montrachet Grand Cru,Le Montrachet,montrachet-grand-cru,NaN,fr,Frankrijk,France,france,EUR,Euros,€,None,1304.0,5552423.0,568341.0,67611.0,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,NaN,NaN,1358,Joseph Drouhin,joseph-drouhin,0,NaN,NaN,NaN,NaN,NaN,3.105895,NaN,3.834148,1.982433,NaN,152.0,132.0,NaN,oak,"[{'id': 70, 'name': 'butter', 'count': 54}, {'...",111.0,13341.0,"[{'id': 6, 'name': 'almond', 'count': 16}, {'i...",earth,"[{'id': 218, 'name': 'honey', 'count': 57}, {'...",103.0,13053.0,"[{'id': 6, 'name': 'almond', 'count': 16}, {'i...",microbio,"[{'id': 123, 'name': 'cream', 'count': 22}, {'...",94.0,63

In [61]:

'wine_taste_flavor_1stats.count','wine_taste_flavor_1stats.score','wine_taste_flavor_1secondary_keywords',
'wine_taste_flavor_2stats.count','wine_taste_flavor_2stats.score','wine_taste_flavor_2secondary_keywords',
'wine_taste_flavor_3stats.count','wine_taste_flavor_3stats.score','wine_taste_flavor_3secondary_keywords',
'wine_taste_flavor_4stats.count','wine_taste_flavor_4stats.score','wine_taste_flavor_4secondary_keywords'


('wine_taste_flavor_4stats.count',
 'wine_taste_flavor_4stats.score',
 'wine_taste_flavor_4secondary_keywords')

#### wine_taste_flavor_keywords

In [62]:
wine_taste_keyword1 = pd.json_normalize(wine_df['wine_taste_flavor_1primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword1_1 = pd.json_normalize(wine_taste_keyword1[0])
wine_taste_keyword1_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword1_1.columns
wine_taste_keyword1_2 = pd.json_normalize(wine_taste_keyword1[1])
wine_taste_keyword1_2.columns = 'wine_taste_flavor_2primary_keywords' + wine_taste_keyword1_2.columns
wine_taste_keyword1_3 = pd.json_normalize(wine_taste_keyword1[2])
wine_taste_keyword1_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword1_3.columns
wine_taste_keyword1_4 = pd.json_normalize(wine_taste_keyword1[3])
wine_taste_keyword1_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword1_4.columns
wine_taste_keyword1_5 = pd.json_normalize(wine_taste_keyword1[4])
wine_taste_keyword1_5.columns = 'wine_taste_flavor_5primary_keywords' + wine_taste_keyword1_5.columns



#combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword1_1, wine_taste_keyword1_2, wine_taste_keyword1_3, wine_taste_keyword1_4, wine_taste_keyword1_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_1primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, 'ra...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,3150.0,Wiltingen,,wiltingen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",NaN,NaN,NaN,2197,Egon Müller - Scharzhof,egon-muller-scharzhof,0,NaN,NaN,NaN,NaN,NaN,4.795868,NaN,3.078479,2.826866,NaN,33.0,14.0,NaN,citrus_fruit,10.0,1500.0,NaN,earth,"[{'id': 218, 'name': 'honey', 'count': 8}, {'i...",10.0,1258.0,NaN,tropical_fruit,"[{'id': 329, 'name': 'pineapple', 'count': 7},...",9.0,1500.0,NaN,tree_fruit,"[{'id': 271, 'name': 'melon', 'count': 1}, {'i...",4.0,500.0,NaN,243.0,lemon,7.0,106.0,citrus,5.0,252.0,lime,1.0,190.0,grapefruit,1.0,411.0,tangerine,1.0
1,1274124,Montrachet Grand Cru Marquis de Laguiche,montrachet-grand-cru-marquis-de-laguiche,2,0,False,"{'id': 3041, 'name': 'Montrachet Grand Cru', '...","{'id': 1358, 'name': 'Joseph Drouhin', 'seo_na...","{'structure': {'acidity': 3.1058955, 'fizzines...","{'status': 'Normal', 'ratings_count': 1870, 'r...","{'id': 285, 'seo_name': 'burgundy-cote-de-beau...",True,3041.0,Montrachet Grand Cru,Le Mont

In [63]:
wine_taste_keyword2 = pd.json_normalize(wine_df['wine_taste_flavor_2primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword2_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword2_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_1.columns
wine_taste_keyword2_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword2_2.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_2.columns
wine_taste_keyword2_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword2_3.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_3.columns
wine_taste_keyword2_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword2_4.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_4.columns
wine_taste_keyword2_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword2_5.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword2_1, wine_taste_keyword2_2, wine_taste_keyword2_3, wine_taste_keyword2_4, wine_taste_keyword2_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_2primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, 'ra...","{'id': 186, 'seo_name': 'german-riesling', 're...",True,3150.0,Wiltingen,,wiltingen,NaN,de,Duitsland,Deutschland,germany,EUR,Euros,€,None,210.0,2383213.0,209807.0,13657.0,"[{'id': 15, 'name': 'Riesling', 'seo_name': 'r...",NaN,NaN,NaN,2197,Egon Müller - Scharzhof,egon-muller-scharzhof,0,NaN,NaN,NaN,NaN,NaN,4.795868,NaN,3.078479,2.826866,NaN,33.0,14.0,NaN,citrus_fruit,10.0,1500.0,NaN,earth,10.0,1258.0,NaN,tropical_fruit,"[{'id': 329, 'name': 'pineapple', 'count': 7},...",9.0,1500.0,NaN,tree_fruit,"[{'id': 271, 'name': 'melon', 'count': 1}, {'i...

In [64]:
wine_taste_keyword3 = pd.json_normalize(wine_df['wine_taste_flavor_3primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword3_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword3_1.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_1.columns
wine_taste_keyword3_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword3_2.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_2.columns
wine_taste_keyword3_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword3_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_3.columns
wine_taste_keyword3_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword3_4.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_4.columns
wine_taste_keyword3_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword3_5.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword3_1, wine_taste_keyword3_2, wine_taste_keyword3_3, wine_taste_keyword3_4, wine_taste_keyword3_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_3primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount
0,1467543,Riesling 'Le Gallais' Wiltinger Braune Kupp Au...,riesling-le-gallais-wiltinger-braune-kupp-auslese,2,0,False,"{'id': 3150, 'name': 'Wiltingen', 'name_en': '...","{'id': 2197, 'name': 'Egon Müller - Scharzhof'...","{'structure': {'acidity': 4.795868, 'fizziness...","{'status': 'Normal', 'ratings_count': 216, '

In [65]:
wine_taste_keyword4 = pd.json_normalize(wine_df['wine_taste_flavor_4primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword4_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword4_1.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_1.columns
wine_taste_keyword4_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword4_2.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_2.columns
wine_taste_keyword4_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword4_3.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_3.columns
wine_taste_keyword4_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword4_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_4.columns
wine_taste_keyword4_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword4_5.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_5.columns


# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword4_1, wine_taste_keyword4_2, wine_taste_keyword4_3, wine_taste_keyword4_4, wine_taste_keyword4_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_4primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_statistics

In [66]:
#Flat columns
wine_statistics = pd.json_normalize(wine_df['wine_statistics'])
wine_statistics.columns = 'wine_statistics_' + wine_statistics.columns
# print(wine_statistics.shape)
# wine_statistics

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_statistics], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 131)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_style

In [67]:
#Flat columns
wine_style = pd.json_normalize(wine_df['wine_style'])
wine_style.columns = 'wine_style_' + wine_style.columns
# print(wine_style.shape)
# wine_style

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_style], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(124975, 190)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

#### wine_style_food

In [68]:
#Flat columns
wine_food = pd.json_normalize(wine_df['wine_style_food'])

#Flat each column from food table
wine_food_1 = pd.json_normalize(wine_food[0])
wine_food_1.columns = 'wine_food_' + wine_food_1.columns
wine_food_2 = pd.json_normalize(wine_food[1])
wine_food_2.columns = 'wine_food_' + wine_food_2.columns
wine_food_3 = pd.json_normalize(wine_food[2])
wine_food_3.columns = 'wine_food_' + wine_food_3.columns
wine_food_4 = pd.json_normalize(wine_food[3])
wine_food_4.columns = 'wine_food_' + wine_food_4.columns
wine_food_5 = pd.json_normalize(wine_food[4])
wine_food_5.columns = 'wine_food_' + wine_food_5.columns


# #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_food_1, wine_food_2, wine_food_3, wine_food_4, wine_food_5], axis=1)
wine_df = wine_df.drop('wine_style_food', axis=1)
wine_df.shape

(124975, 214)

#### wine_style_grapes

In [69]:
#Flat columns
wine_grapes = pd.json_normalize(wine_df['wine_style_grapes'])

# #Flat each column from food table
wine_grapes_1 = pd.json_normalize(wine_grapes[0])
wine_grapes_1.columns = 'wine_style_grapes' + wine_grapes_1.columns
wine_grapes_2 = pd.json_normalize(wine_grapes[1])
wine_grapes_2.columns = 'wine_style_grapes_' + wine_grapes_2.columns
wine_grapes_3 = pd.json_normalize(wine_grapes[2])
wine_grapes_3.columns = 'wine_style_grapes_' + wine_grapes_3.columns
wine_grapes_4 = pd.json_normalize(wine_grapes[3])
wine_grapes_4.columns = 'wine_style_grapes_' + wine_grapes_4.columns
wine_grapes_5 = pd.json_normalize(wine_grapes[4])
wine_grapes_5.columns = 'wine_style_grapes_' + wine_grapes_5.columns
wine_grapes_6 = pd.json_normalize(wine_grapes[5])
wine_grapes_6.columns = 'wine_style_grapes_' + wine_grapes_6.columns


# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_grapes_1, wine_grapes_2, wine_grapes_3, wine_grapes_4, wine_grapes_5, wine_grapes_6], axis=1)
wine_df = wine_df.drop('wine_style_grapes', axis=1)
wine_df.shape

(124975, 243)

## Price

In [70]:
all_wines = []
prices = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        prices.append(all_wines[i][j]['price'])

In [71]:
prices_df = pd.DataFrame(prices)
prices_df.head(3)

,id,amount,discounted_from,discount_percent,type,sku,url,visibility,bottle_type_id,currency,bottle_type
0,22634611,285.40,NaN,NaN,vc,a2d08d2e-f4f7-4431-b7a7-bc31479261be,https://hermanwines.com/products/wilting-br-ku...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
1,27302625,811.80,NaN,NaN,vc,BCH20083000162,https://www.wijnig.nl/joseph-drouhin-montrache...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
2,24576883,349.13,NaN,NaN,vc,93342,http://cultwijnen.nl/,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."


#### Remove unnecessary columns

Price information will be used to improve recommendatio. Only the price per bottle is relevant

In [72]:
drop_lst = ['id','type','discounted_from','discount_percent','sku','url','visibility','bottle_type_id','bottle_type']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

,amount,currency
0,285.40,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
1,811.80,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
2,349.13,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
3,225.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
4,295.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
...,...,...
124970,26.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
124971,20.88,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
124972,32.05,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
124973,39.90,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."


In [73]:
#Flat columns
currency = pd.json_normalize(prices_df['currency'])

# # #combine all dfs. drop original column, keep flattened
prices_df = pd.concat([prices_df, currency], axis=1)
prices_df = prices_df.drop('currency', axis=1)
prices_df.shape

# keep only currency code
drop_lst = ['name','prefix','suffix']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

,amount,code
0,285.40,EUR
1,811.80,EUR
2,349.13,EUR
3,225.00,EUR
4,295.00,EUR
...,...,...
124970,26.00,EUR
124971,20.88,EUR
124972,32.05,EUR
124973,39.90,EUR


## Make full df from all tables above

In [74]:
all_wines_df = pd.concat([vintage_df,prices_df,statistics_df,wine_df], axis = 1)
print(all_wines_df.shape)
all_wines_df.head(3)

(124975, 251)


,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor

In [75]:
len(all_wines_df)

124975

In [76]:
columns_to_drop = ['wine_region','wine_winery','wine_taste','wine_statistics','wine_style']

In [77]:
all_wines_df = all_wines_df.drop(columns_to_drop, axis = 1)
all_wines_df

,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_background_image,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_2group,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_3group,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_4group,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscoun

In [78]:
all_wines_df.to_csv('all_wines_df_type2_v3.csv')